# Organize the Data

In [1]:
import numpy as np

In [2]:
#Create references to important directories we will use over and over
import os, sys

LESSON_HOME_DIR = '/home/paperspace/NBs/Cervical_Cancer_Comp'
DATA_HOME_DIR = LESSON_HOME_DIR+'/data/'

In [3]:
#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

## Create validation set and sample

In [4]:
#Create directories
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results

/home/paperspace/NBs/Cervical_Cancer_Comp/data
mkdir: cannot create directory ‘valid’: File exists
mkdir: cannot create directory ‘results’: File exists


In [5]:
#creating respecting class dirs in valid
%cd valid/
%ls
%mkdir Type_1
%mkdir Type_2
%mkdir Type_3

/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid


In [7]:
# current dir
%pwd

u'/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid'

In [8]:
%cd $DATA_HOME_DIR/train

/home/paperspace/NBs/Cervical_Cancer_Comp/data/train


In [9]:
from glob import glob

In [10]:
%ls
g = glob('**/*.jpg')
shuf = np.random.permutation(g)
validation_dir = DATA_HOME_DIR+'valid/' 

for i in range(444): os.rename(shuf[i], validation_dir + shuf[i])

results/  Type_1/  Type_2/  Type_3/  valid/


In [16]:
# check valid dir for type 1
%cd $DATA_HOME_DIR
%ls
%cd valid/Type_1
%ls -1 | wc -l

/home/paperspace/NBs/Cervical_Cancer_Comp/data
bak/  download.sh*  results/  test/  test.7z  train/  train.7z  valid/
/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid/Type_1
76


In [17]:
# check valid dir for type 2
%cd ..
%cd Type_2
% ls -1 | wc -l

/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid
/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid/Type_2
229


In [18]:
# check valid dir for type 2
%cd ..
%cd Type_3
% ls -1 | wc -l

/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid
/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid/Type_3
139


# Creating the Model

In [ ]:
# imports
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras import backend as k

In [ ]:
train_data_dir = 'data/train'
test_data_dir = 'data/test'
# validation_data_dir = 'data/validation'


nb_train_samples = 1037
nb_validation_samples = 444

if we are using Inception or Xception, we need to set the inputShape  to 299×299 pixels, followed by updating preprocess  to use a separate pre-processing function that performs a different type of scaling.

In [ ]:
# hyper parameters for model
nb_classes = 3  # number of classes
img_width, img_height = 299, 299  # change based on the shape/structure of your images
batch_size = 16  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 1  # number of iteration the algorithm gets trained. 1 As test run
learn_rate = 0.045  # sgd learning rate
momentum = 0.9  # sgd momentum to avoid local minimum
transformation_ratio = .05  # how aggressive will be the data augmentation/transformation

### Loading Pre-trained InceptionV3 Model using imagenet dataset weight

In [ ]:
# Pre-Trained CNN Model using imagenet dataset for pre-trained weights
base_model = InceptionV3(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
print('Model Loaded!')

## Defining own top model block

In [ ]:
# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(nb_classes, activation='relu')(x)

In [ ]:
# add your top layer block to your base model
model = Model(base_model.input, predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all layers of the based model that is already pre-trained.
for layer in base_model.layers:
    layer.trainable = False